In [ ]:
#load data

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

data = pd.read_csv(".../compas-scores-two-years.csv")
data = pd.DataFrame(data)

data_w = data[data['race'] == 'Caucasian']
data_b = data[data['race'] == 'African-American']

data = pd.concat([data_w, data_b])

for col in set(data.columns) - set(data.describe().columns):
  data[col] = data[col].astype('category')
 
def clip(df, a):
  perm = np.random.permutation(df.index)
  m = len(df.index)
  clip_end = int(a*m)
  cliped = df.iloc[:clip_end]
  return cliped

def oneHotCatVars(df, df_cols):
    
    df_1 = df.drop(columns = df_cols, axis = 1)
    df_2 = pd.get_dummies(df[df_cols])
    
    return (pd.concat([df_1, df_2], axis=1, join='inner'))

data_preprocessed = oneHotCatVars(data, data.select_dtypes('category').columns)

 
normalize_columns = ['age', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count']
 
def normalize(columns):
  scaler = preprocessing.StandardScaler()
  data_preprocessed[columns] = scaler.fit_transform(data_preprocessed[columns])
 
normalize(normalize_columns)

x_train, x_test = train_test_split(data_preprocessed)

b_test = x_test[x_test['race_African-American'] == 1]
w_test = x_test[x_test['race_Caucasian'] == 1]

train = x_train.drop(['two_year_recid'],axis=1)
train = train.drop(['race_African-American'],axis=1)
train = train.drop(['race_Caucasian'],axis=1)
train_label = x_train['two_year_recid']
test_x = x_test.drop(['two_year_recid'],axis=1)
test_x = test_x.drop(['race_African-American'],axis=1)
test_x = test_x.drop(['race_Caucasian'],axis=1)
test_label = x_test['two_year_recid']
b_test_x = b_test.drop(['two_year_recid'],axis=1)
b_test_x = b_test_x.drop(['race_African-American'],axis=1)
b_test_x = b_test_x.drop(['race_Caucasian'],axis=1)
b_test_y = b_test['two_year_recid']
w_test_x = w_test.drop(['two_year_recid'],axis=1)
w_test_x = w_test_x.drop(['race_African-American'],axis=1)
w_test_x = w_test_x.drop(['race_Caucasian'],axis=1)
w_test_y = w_test['two_year_recid']

In [ ]:
#vanilla classifier

def model_eval(actual, pred):
    
    confusion = pd.crosstab(actual, pred, rownames=['Actual'], colnames=['Predicted'])
    TP = confusion.loc[1,1]
    TN = confusion.loc[0,0]
    FP = confusion.loc[0,1]
    FN = confusion.loc[1,0]
    
    out = {}
    out['ALL'] = (TP+TN+FP+FN)
    out['DP'] = (TP+FP)/(TP+TN+FP+FN)
    out['TPR'] =  TP/(TP+FN)
    out['TNR'] = TN/(FP+TN)
    out['FPR'] = FP/(FP+TN)
    out['FNR'] = FN/(TP+FN)
    out['ACR'] = (TP+TN)/(TP+TN+FP+FN)
    
    return out

log_reg = LogisticRegression(penalty = 'l2', dual = False, tol = 1e-4, fit_intercept = False, 
                            max_iter=400, solver='newton-cg', warm_start = True)
log_reg.fit(train, train_label)

log_reg_pred1 = log_reg.predict(b_test_x)
logistic_reg1 = model_eval(b_test_y, log_reg_pred1)
ovl_logreg1 = round(pd.DataFrame([logistic_reg1], index = ['logistic_reg_Black']),4)
display(ovl_logreg1)

log_reg_pred2 = log_reg.predict(w_test_x)
logistic_reg2 = model_eval(w_test_y, log_reg_pred2)
ovl_logreg2 = round(pd.DataFrame([logistic_reg2], index = ['logistic_reg_White']),4)
display(ovl_logreg2)

log_reg_pred3 = log_reg.predict(test_x)
logistic_reg3 = model_eval(test_label, log_reg_pred3)
ovl_logreg3 = round(pd.DataFrame([logistic_reg3], index = ['logistic_reg_All']),4)
display(ovl_logreg3)

DI = round(100 * abs(logistic_reg2['DP'] - logistic_reg1['DP']), 4)
DFPR = round(100 * abs(logistic_reg2['TNR'] - logistic_reg1['TNR']), 4)
DFNR = round(100 * abs(logistic_reg2['TPR'] - logistic_reg1['TPR']), 4)
prule = round(100 * min(logistic_reg2['DP'] / logistic_reg1['DP'], logistic_reg1['DP'] / logistic_reg2['DP']), 4)
ACR = round(logistic_reg3['ACR']*100, 4)

print(f'Baseline method: Average accuracy is {ACR}%, Disparate impact is {DI}%, disparate TPR is {DFNR}%,\n disparate TNR is {DFPR}%, p% rule is {prule}%.')

In [ ]:
from sklearn.metrics import log_loss
import math
import cvxpy as cp

wi0 = np.ones((x_train.shape[0]))
wi1 = wi0 + 10000000000000


x_train_b = x_train[x_train['race_African-American'] == 1]
x_train_bp = x_train_b[x_train_b['two_year_recid'] == 1]
x_train_bn = x_train_b[x_train_b['two_year_recid'] == 0]
x_train_w = x_train[x_train['race_Caucasian'] == 1]
x_train_wp = x_train_w[x_train_w['two_year_recid'] == 1]
x_train_wn = x_train_w[x_train_w['two_year_recid'] == 0]
train_bp = x_train_bp.drop(['two_year_recid'],axis=1)
train_bp = train_bp.drop(['race_African-American'],axis=1)
train_bp = train_bp.drop(['race_Caucasian'],axis=1)
bp_label = x_train_bp['two_year_recid'].to_numpy()
train_bn = x_train_bn.drop(['two_year_recid'],axis=1)
train_bn = train_bn.drop(['race_African-American'],axis=1)
train_bn = train_bn.drop(['race_Caucasian'],axis=1)
bn_label = x_train_bn['two_year_recid'].to_numpy()
train_wp = x_train_wp.drop(['two_year_recid'],axis=1)
train_wp = train_wp.drop(['race_African-American'],axis=1)
train_wp = train_wp.drop(['race_Caucasian'],axis=1)
wp_label = x_train_wp['two_year_recid'].to_numpy()
train_wn = x_train_wn.drop(['two_year_recid'],axis=1)
train_wn = train_wn.drop(['race_African-American'],axis=1)
train_wn = train_wn.drop(['race_Caucasian'],axis=1)
wn_label = x_train_wn['two_year_recid'].to_numpy()

loss_bp = wi0_bp = np.ones(x_train_bp.shape[0])
loss_bn = wi0_bn = np.ones(x_train_bn.shape[0])
loss_wp = wi0_wp = np.ones(x_train_wp.shape[0])
loss_wn = wi0_wn = np.ones(x_train_wn.shape[0])

def optim(loss, a, c):
  A = loss
  x = cp.Variable(loss.shape[0])
  objective = cp.Maximize(-a*cp.sum_squares(x)+cp.sum(cp.multiply(A,x)))
  constraints = [0 <= x, cp.sum(x) == c]
  prob = cp.Problem(objective, constraints)   
  result = prob.solve()
  for i in range(x.value.shape[0]):
    if abs(x.value[i]) < 0.01 or x.value[i] < 0:
      x.value[i] = 0
  x.value = x.value
  return x.value

def model_eval(actual, pred):
    
    confusion = pd.crosstab(actual, pred, rownames=['Actual'], colnames=['Predicted'])
    TP = confusion.loc[1,1]
    TN = confusion.loc[0,0]
    FP = confusion.loc[0,1]
    FN = confusion.loc[1,0]
    
    out = {}
    out['ALL'] = (TP+TN+FP+FN)
    out['DP'] = (TP+FP)/(TP+TN+FP+FN)
    out['TPR'] =  TP/(TP+FN)
    out['TNR'] = TN/(FP+TN)
    out['FPR'] = FP/(FP+TN)
    out['FNR'] = FN/(TP+FN)
    out['ACR'] = (TP+TN)/(TP+TN+FP+FN)
    
    return out

def dif(a, b):
  sum = 0
  for i in range(len(a)):
    sum += (a[i] - b[i]) ** 2
  sum0 = sum ** 0.5
  return sum0

In [ ]:
def Random_search(train, folds, param_range, num_param = 10):
    trainscore = []
    testscore = []
    K_values = sample(range(param_range[0],param_range[1]),num_param)
    K_values.sort()
    print('k range:',K_values)
    for k in tqdm(K_values):
        trainscore_folds = []
        testscore_folds = []
        for j in range(0, folds):
            x_train, x_test = train_test_split(train)
            Y_train = x_train['two_year_recid'].to_numpy()
            X_train = x_train.drop(['two_year_recid'],axis=1)
            X_train = X_train.drop(['race_African-American'],axis=1)
            X_train = X_train.drop(['race_Caucasian'],axis=1).to_numpy()
            Y_test = x_test['two_year_recid'].to_numpy()
            X_test = x_test.drop(['two_year_recid'],axis=1)
            X_test = X_test.drop(['race_African-American'],axis=1)
            X_test = X_test.drop(['race_Caucasian'],axis=1).to_numpy()

            iter = 0
            wi0 = np.ones((X_train.shape[0]))
            wi1 = wi0 + 10000000000000
            x_train_b = x_train[x_train['race_African-American'] == 1]
            x_train_bp = x_train_b[x_train_b['two_year_recid'] == 1]
            x_train_bn = x_train_b[x_train_b['two_year_recid'] == 0]
            x_train_w = x_train[x_train['race_Caucasian'] == 1]
            x_train_wp = x_train_w[x_train_w['two_year_recid'] == 1]
            x_train_wn = x_train_w[x_train_w['two_year_recid'] == 0]
            train_bp = x_train_bp.drop(['two_year_recid'],axis=1)
            train_bp = train_bp.drop(['race_African-American'],axis=1)
            train_bp = train_bp.drop(['race_Caucasian'],axis=1).to_numpy()
            bp_label = x_train_bp['two_year_recid'].to_numpy()
            train_bn = x_train_bn.drop(['two_year_recid'],axis=1)
            train_bn = train_bn.drop(['race_African-American'],axis=1)
            train_bn = train_bn.drop(['race_Caucasian'],axis=1).to_numpy()
            bn_label = x_train_bn['two_year_recid'].to_numpy()
            train_wp = x_train_wp.drop(['two_year_recid'],axis=1)
            train_wp = train_wp.drop(['race_African-American'],axis=1)
            train_wp = train_wp.drop(['race_Caucasian'],axis=1).to_numpy()
            wp_label = x_train_wp['two_year_recid'].to_numpy()
            train_wn = x_train_wn.drop(['two_year_recid'],axis=1)
            train_wn = train_wn.drop(['race_African-American'],axis=1)
            train_wn = train_wn.drop(['race_Caucasian'],axis=1).to_numpy()
            wn_label = x_train_wn['two_year_recid'].to_numpy()

            loss_bp = wi0_bp = np.ones(x_train_bp.shape[0])
            loss_bn = wi0_bn = np.ones(x_train_bn.shape[0])
            loss_wp = wi0_wp = np.ones(x_train_wp.shape[0])
            loss_wn = wi0_wn = np.ones(x_train_wn.shape[0])

            while dif(wi0, wi1) > 0.0001:
              classifier = LogisticRegression(penalty = 'none', dual = False, tol = 1e-4, fit_intercept = False, 
                            max_iter=400, solver='newton-cg', warm_start = True)
              classifier.fit(X_train, Y_train, wi0)

              for i1 in range(train_bp.shape[0]):
                loss_bp[i1] = -math.log(log_reg.predict_proba(train_bp)[i1][1]+1e-100)

              for i2 in range(train_bn.shape[0]):
                loss_bn[i2] = -math.log(log_reg.predict_proba(train_bn)[i2][0]+1e-100)

              for i3 in range(train_wp.shape[0]):
                loss_wp[i3] = -math.log(log_reg.predict_proba(train_wp)[i3][1]+1e-100)

              for i4 in range(train_wn.shape[0]):
                loss_wn[i4] = -math.log(log_reg.predict_proba(train_wn)[i4][0]+1e-100)

              wi1 = wi0
              wi0_bp_1 = optim(loss_bp, k, c)
              wi0_bn_1 = optim(loss_bn, k, c)
              wi0_wp_1 = optim(loss_wp, k, c)
              wi0_wn_1 = optim(loss_wn, k, c)

              X_train = np.concatenate([train_bp, train_bn, train_wp, train_wn])
              Y_train = np.concatenate((bp_label, bn_label, wp_label, wn_label))
              wi0 = np.concatenate((wi0_bp_1, wi0_bn_1, wi0_wp_1, wi0_wn_1))

              iter = iter + 1


            Y_predicted = classifier.predict(X_test)
            testscore_folds.append(np.sum(Y_test == Y_predicted)/len(Y_test))

            Y_predicted = classifier.predict(X_train)
            trainscore_folds.append(np.sum(Y_train == Y_predicted)/len(Y_train))
        trainscore.append(np.mean(np.array(trainscore_folds)))
        testscore.append(np.mean(np.array(testscore_folds)))
    return trainscore, testscore , K_values

In [ ]:
a = np.ones(10)
b = np.ones(10)
np.sum(a == b)

10

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import random
import warnings
from tqdm import tqdm

warnings.filterwarnings("ignore")

from random import sample 

param_min = 1 
param_max = 20 
k_fold = 5
param_range = (param_min, param_max) 
trainscore, testscore, K_values = Random_search(x_train, k_fold, param_range)

In [ ]:
iter = 0

while dif(wi0, wi1) > 0.0001:
  log_reg = LogisticRegression(penalty = 'none', dual = False, tol = 1e-4, fit_intercept = False, 
                            max_iter=400, solver='newton-cg', warm_start = True)
  log_reg.fit(train, train_label, wi0)

  for i1 in range(train_bp.shape[0]):
    loss_bp[i1] = -math.log(log_reg.predict_proba(train_bp)[i1][1]+1e-100)

  for i2 in range(train_bn.shape[0]):
    loss_bn[i2] = -math.log(log_reg.predict_proba(train_bn)[i2][0]+1e-100)

  for i3 in range(train_wp.shape[0]):
    loss_wp[i3] = -math.log(log_reg.predict_proba(train_wp)[i3][1]+1e-100)

  for i4 in range(train_wn.shape[0]):
    loss_wn[i4] = -math.log(log_reg.predict_proba(train_wn)[i4][0]+1e-100)

  wi1 = wi0
  wi0_bp_1 = optim(loss_bp, a, c)
  wi0_bn_1 = optim(loss_bn, a, c)
  wi0_wp_1 = optim(loss_wp, a, c)
  wi0_wn_1 = optim(loss_wn, a, c)

  train = pd.concat([train_bp, train_bn, train_wp, train_wn])
  train_label = np.concatenate((bp_label, bn_label, wp_label, wn_label))
  wi0 = np.concatenate((wi0_bp_1, wi0_bn_1, wi0_wp_1, wi0_wn_1))

  iter = iter + 1


  log_reg_pred1 = log_reg.predict(b_test_x)
  logistic_reg1 = model_eval(b_test_y, log_reg_pred1)
  ovl_logreg1 = round(pd.DataFrame([logistic_reg1], index = ['logistic_reg_Black']),4)
  display(ovl_logreg1)

  log_reg_pred2 = log_reg.predict(w_test_x)
  logistic_reg2 = model_eval(w_test_y, log_reg_pred2)
  ovl_logreg2 = round(pd.DataFrame([logistic_reg2], index = ['logistic_reg_White']),4)
  display(ovl_logreg2)

  log_reg_pred3 = log_reg.predict(test_x)
  logistic_reg3 = model_eval(test_label, log_reg_pred3)
  ovl_logreg3 = round(pd.DataFrame([logistic_reg3], index = ['logistic_reg_All']),4)
  display(ovl_logreg3)

  DI = 100 * abs(logistic_reg2['DP'] - logistic_reg1['DP'])
  DFPR = 100 * abs(logistic_reg2['TNR'] - logistic_reg1['TNR'])
  DFNR = 100 * abs(logistic_reg2['TPR'] - logistic_reg1['TPR'])
  prule = 100 * min(logistic_reg2['DP'] / logistic_reg1['DP'], logistic_reg1['DP'] / logistic_reg2['DP'])

  print(f'Our method: Iteration {iter-1}, disparate impact is {DI}%, disparate FPR is {DFPR}%,\n disparate FNR is {DFNR}%, p% rule is {prule}%，\n disparate accuracy is {DACR}%.')

log_reg_pred1 = log_reg.predict(b_test_x)
logistic_reg1 = model_eval(b_test_y, log_reg_pred1)
ovl_logreg1 = round(pd.DataFrame([logistic_reg1], index = ['logistic_reg_Black']),4)
display(ovl_logreg1)

log_reg_pred2 = log_reg.predict(w_test_x)
logistic_reg2 = model_eval(w_test_y, log_reg_pred2)
ovl_logreg2 = round(pd.DataFrame([logistic_reg2], index = ['logistic_reg_White']),4)
display(ovl_logreg2)

log_reg_pred3 = log_reg.predict(test_x)
logistic_reg3 = model_eval(test_label, log_reg_pred3)
ovl_logreg3 = round(pd.DataFrame([logistic_reg3], index = ['logistic_reg_All']),4)
display(ovl_logreg3)

DI = 100 * abs(logistic_reg2['DP'] - logistic_reg1['DP'])
DFPR = 100 * abs(logistic_reg2['TNR'] - logistic_reg1['TNR'])
DFNR = 100 * abs(logistic_reg2['TPR'] - logistic_reg1['TPR'])
prule = 100 * min(logistic_reg2['DP'] / logistic_reg1['DP'], logistic_reg1['DP'] / logistic_reg2['DP'])

print(f'Our method: Disparate impact is {DI}%, disparate FPR is {DFPR}%,\n disparate FNR is {DFNR}%, p% rule is {prule}%.')